In [1]:
from src.models.model import SwarModel
from src.file_format.fileformat import BitPacker, BitUnpacker
import torch
import io

In [2]:
from src.file_format.fileformat import read_nac_header

In [3]:
model = SwarModel.build_model(24).to(torch.device('cuda'))

path = "./models/neural-audio-codec.pt"

checkpoint = torch.load(path)
model.load_state_dict(checkpoint["model_state_dict"])

<All keys matched successfully>

In [4]:
import torchaudio

wf, sr = torchaudio.load("pianon.wav")

model.set_target_bandwidth(24)   # target bitrate is 24kbps

In [5]:
wf = wf.view(1, 1, wf.shape[-1]).to(torch.device('cuda'))

In [6]:
# compressing the long audio file in parts of 2s clip

audio_wav = torch.empty(0).to(torch.device('cuda')).to(torch.int32)

for i in range(0, 26):
    offset = i * 32000
    audio = wf[:, :, offset:offset+32000]
    print(audio.shape)
    output = model._encode_frame(audio)[0]
    audio_wav = torch.cat([audio_wav, torch.flatten(output)], dim=0)
        
output = audio_wav.tolist()

torch.Size([1, 1, 32000])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 32000])


In [7]:
output

[355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 355,
 131,
 903,
 131,
 903,
 551,
 849,
 216,
 118,
 593,
 551,
 935,
 216,
 156,
 593,
 551,
 935,
 769,
 156,
 903,
 296,
 849,
 296,
 842,
 414,
 298,
 992,
 340,
 224,
 387,
 433,
 298,
 340,
 515,
 3,
 547,
 433,
 298,
 340,
 224,
 3,
 547,
 298,
 123,
 134,
 977,
 977,
 977,
 977,
 981,
 981,
 977,
 981,
 977,
 977,
 977,
 977,
 981,
 981,
 977,
 977,
 981,
 977,
 977,
 977,
 977,
 977,
 981,
 977,
 977,
 977,
 977,
 977,
 977,
 977,
 977,
 977,
 977,
 977,
 977,
 977,
 977,
 977,
 977,
 977,
 977,
 981,
 977,
 977,
 981,
 981,
 977,
 977,
 981,
 977,
 977,
 977,
 981,
 977,
 969,
 729,
 242,
 770,
 930,
 478,
 927,
 948,
 656,
 991,
 981,
 709,
 292,
 6

In [8]:
from src.file_format.fileformat import *

In [13]:
fo = io.BytesIO()
write_nac_header(fo, 3, None)
packer = BitPacker(10, fo)

for token in output:
    packer.push(token)

packer.flush()

In [14]:
# saving compressed audio data into the custom file format

with open('pianon.nac', 'wb') as outputfile:
    outputfile.write(fo.getbuffer())

In [13]:
# decoding the .nac file for playing

with open('8kn.nac', 'rb') as inputfile:
    buffer = inputfile.read()
    fo = io.BytesIO(buffer)

In [14]:
fo.seek(0)

0

In [15]:
read_nac_header(fo)

(24, None)

In [16]:
unpacker = BitUnpacker(10, fo)
rebuilt = []

In [17]:
while True:
    value = unpacker.pull()
    if value is None:
        break
    rebuilt.append(value)

In [18]:
decompressed_output = torch.Tensor(rebuilt).int()

In [19]:
decompressed_output.shape

torch.Size([124800])

In [20]:
wav = torch.empty(1, 0).to(torch.device('cuda'))
for i in range(0, 26):
    offset = i * 4800
    data = decompressed_output[offset:offset+4800].to(torch.device('cuda'))
    output = model._decode_frame(data.view(1, 48, 100).long())
    wav = torch.cat([wav, output.view(1, 32000)], dim=1)

In [21]:
from IPython.display import Audio
Audio(wav.cpu().detach(), rate=16000)